In [1]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import torch
from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# 기계 번역에 사용할 데이터셋은 왼쪽의 영어 문장과 오른쪽의 프랑스어 문장을 tab으로 구분한 형식(19만개의 병렬 문장 샘플)
!wget -c http://www.manythings.org/anki/fra-eng.zip && unzip -o fra-eng.zip

--2024-08-30 00:48:01--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  17.3MB/s    in 0.4s    

2024-08-30 00:48:02 (17.3 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [3]:
def unicode_to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [4]:
def preprocess_sentence(sent):
  # 악센트 삭제 함수 호출
  sent = unicode_to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백을 만듭니다.
  # Ex) "he is a boy." => "he is a boy ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [5]:
num_samples = 33000
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t') # tab을 기준으로 source(영어)와 target(프랑스어)로 구분

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()] # Decoder의 입력(프랑스어 입력)으로는 <sos>가 맨 앞에 추가된다
      tar_line_out = [w for w in (tar_line + " <eos>").split()] # Decoder의 출력(프랑스어 출력)으로는 <eos>가 마지막에 추가된다

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target

In [6]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [7]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()

In [8]:
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


In [9]:
def build_vocab(sents):
  # 사전 만들기
  word_list = []
  for sent in sents:
      for word in sent:
        word_list.append(word)

  # 각 단어별 등장 빈도를 계산하여 등장 빈도가 높은 순서로 정렬
  word_counts = Counter(word_list)
  vocab = sorted(word_counts, key=word_counts.get, reverse=True)

  # 사전을 바탕으로 단어를 정수 인덱스로 인코딩하는 딕셔너리 생성
  word_to_index = {}
  word_to_index['<PAD>'] = 0
  word_to_index['<UNK>'] = 1

  # 등장 빈도가 높은 단어일수록 낮은 정수를 부여
  for index, word in enumerate(vocab) :
    word_to_index[word] = index + 2

  return word_to_index

In [10]:
src_vocab = build_vocab(sents_en_in)
tar_vocab = build_vocab(sents_fra_in + sents_fra_out) # 프랑스어 입력과 프랑스어 출력 문장들로 vocab을 만들고 정수 인덱스 인코딩을 만든다

src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4486, 프랑스어 단어 집합의 크기 : 7879


In [11]:
# 영어와 프랑스어 모두 word_to_index의 반대인 정수 인덱스로부터 원래의 문장을 복원(Decoding)하는 딕셔너리 생성
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

def texts_to_sequences(sents, word_to_index):
  encoded_X_data = []
  for sent in tqdm(sents):
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError: # 사전에 없는 단어는 Unk 토큰의 정수 인덱스(1번)으로 변환
          index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

In [12]:
encoder_input = texts_to_sequences(sents_en_in, src_vocab)
decoder_input = texts_to_sequences(sents_fra_in, tar_vocab)
decoder_target = texts_to_sequences(sents_fra_out, tar_vocab)

100%|██████████| 33000/33000 [00:00<00:00, 127302.12it/s]


In [13]:
# 상위 5개의 샘플에 대해서 정수 인코딩 전, 후 문장 출력
# 인코더 입력이므로 <sos>나 <eos>가 없음
for i, (item1, item2) in zip(range(5), zip(sents_en_in, encoder_input)):
    print(f"Index: {i}, 정수 인코딩 전: {item1}, 정수 인코딩 후: {item2}")

Index: 0, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 1, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 2, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 3, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 4, 정수 인코딩 전: ['hi', '.'], 정수 인코딩 후: [736, 2]


In [14]:
def pad_sequences(sentences, max_len=None):
    # 최대 길이 값이 주어지지 않을 경우 데이터 내 최대 길이로 패딩
    if max_len is None:
        max_len = max([len(sentence) for sentence in sentences])

    # 최대길이까지 0으로 초기화(0은 PAD 토큰)
    features = np.zeros((len(sentences), max_len), dtype=int)
    for index, sentence in enumerate(sentences):
        if len(sentence) != 0:
            features[index, :len(sentence)] = np.array(sentence)[:max_len] # 최대 길이만큼 기존 문장의 인코딩된 정수 인덱스 sequence를 덮씌운다
    return features

In [15]:
# 문장들을 정수 인덱스들의 시퀀스로 변환해둔 상태
# 이번에는 padding을 추가하여 길이를 맞춰준다
encoder_input = pad_sequences(encoder_input)
decoder_input = pad_sequences(decoder_input)
decoder_target = pad_sequences(decoder_target)

print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (33000, 7)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


In [16]:
indices = np.arange(encoder_input.shape[0]) # 33000을 범위로 하는 index 리스트를 만들고
np.random.shuffle(indices)                  # shuffle해서 뒤섞는다
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [19443 10350  3505 ... 31503  7147   488]


In [17]:
# 뒤섞은 index를 이용해서 encoder_input, decoder_input, decoder_target을 모두 뒤섞어준다
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

# 랜덤한 인덱스에 대해 값을 찍어본 결과
print([index_to_src[word] for word in encoder_input[30997]]) # encoder의 입력(영어 문장)
print([index_to_tar[word] for word in decoder_input[30997]]) # decoder의 입력(<sos>로 시작하는 프랑스어 문장)
print([index_to_tar[word] for word in decoder_target[30997]]) # decoder의 출력(<eos>로 끝나는 프랑스어 문장)

['tom', 'was', 'invited', '.', '<PAD>', '<PAD>', '<PAD>']
['<sos>', 'tom', 'a', 'ete', 'invite', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['tom', 'a', 'ete', 'invite', '.', '<eos>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [18]:
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [19]:
# 뒤에서 3300개 전까지를 train으로 잡고
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

# 뒤의 3300개를 test로 잡는다
encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [20]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 7)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 7)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


## Machine Translation (Training)

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim


embedding_dim = 256 # Seq2Seq를 이루는 LSTM에 입력으로 들어가기 전 단어를 256차원의 임베딩 값으로 변환
hidden_units = 256  # LSTM의 각 time step의 hidden state의 dimension(=hidden_size)

class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_dim, hidden_units):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(src_vocab_size, embedding_dim, padding_idx=0) # 0번 인덱스를 padding으로
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)

    def forward(self, x):
        # x.shape == (batch_size, seq_len, embedding_dim)
        x = self.embedding(x)
        # hidden.shape == (1, batch_size, hidden_units), cell.shape == (1, batch_size, hidden_units)
        _, (hidden, cell) = self.lstm(x)
        # 인코더의 출력은 hidden state, cell state
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, tar_vocab_size, embedding_dim, hidden_units):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(tar_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)

        # lstm의 마지막 layer의 출력 dim은 hidden state의 dim(=hidden size)로 이것이 출력층인 Fully Connected의 입력이 되고
        # 출력층의 출력은 target인 프랑스어의 어휘 개수가 된다
        self.fc = nn.Linear(hidden_units, tar_vocab_size)

    def forward(self, x, hidden, cell):
        x = self.embedding(x) # x.shape == (batch_size, seq_len, embedding_dim)

        # 디코더의 LSTM으로 인코더의 hidden state, cell state를 전달.
        # output.shape == (batch_size, seq_len, hidden_units)
        # hidden.shape == (1, batch_size, hidden_units)
        # cell.shape == (1, batch_size, hidden_units)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))

        # output.shape: (batch_size, seq_len, tar_vocab_size)
        output = self.fc(output)

        # 디코더의 출력은 예측값, hidden state, cell state
        return output, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        hidden, cell = self.encoder(src)    # Encoder를 통해 hidden state와 cell state를 계산하고
                                            # 그 hidden이 Decoder의 첫 입력에 주어진다
        output, _, _ = self.decoder(trg, hidden, cell) # 훈련 중에는 디코더의 출력 중 오직 output만 사용(Teacher Forcing으로 다음 입력은 주어진다)
        return output

encoder = Encoder(src_vocab_size, embedding_dim, hidden_units)
decoder = Decoder(tar_vocab_size, embedding_dim, hidden_units)
model = Seq2Seq(encoder, decoder)

loss_function = nn.CrossEntropyLoss(ignore_index=0) # PAD 토큰의 정수 인덱스인 0번에 대한 손실 계산은 무시
optimizer = optim.Adam(model.parameters())

In [22]:
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4486, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(7879, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=7879, bias=True)
  )
)


- 영어 문장은 Encoder에 입력되는데 영어 문장들을 토크나이징하여 얻은 영어 사전의 어휘 개수인 4486이 원핫인코딩된 정수 인덱스를 의미하며 이것이 embedding 차원인 256으로 변환된다
- Encoder에서 LSTM에 입력으로 임베딩된 256차원의 입력이 들어와서 256차원의 hidden state의 dimension이 반환된다
- Decoder에서는 Encoder의 hidden state와 함께 target인 프랑스 문장을 정수 인코딩했다가 256차원의 임베딩된 값을 입력 받는다
- Decoder의 LSTM에 256차원의 입력이 들어오고 이를 256차원의 hidden state로 반환한다
- Decoder에서 매 time step마다의 hidden state를 Fully Connected Layer에 집어넣어서 target인 프랑스 사전의 어휘 개수인 7879차원에 맞게 반환한다
- 각 차원에 있는 로짓값은 해당 time step에서 계산된 hidden state를 바탕으로 프랑스어 사전에서 각 단어에 대한 로짓값이 된다

In [23]:
def evaluation(model, dataloader, loss_function, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_count = 0

    # gradient 계산 및 업데이트 없이
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, decoder_targets in dataloader:
            # 데이터를 꺼내오고
            encoder_inputs = encoder_inputs.to(device)
            decoder_inputs = decoder_inputs.to(device)
            decoder_targets = decoder_targets.to(device)

            # Forward Pass
            outputs = model(encoder_inputs, decoder_inputs) # outputs.shape == (batch_size, seq_len, tar_vocab_size)

            # 손실 계산
            # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
            # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
            loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
            total_loss += loss.item()

            # 정확도 계산
            mask = decoder_targets != 0 # 패딩 토큰 제외
            total_correct += ((outputs.argmax(dim=-1) == decoder_targets) * mask).sum().item() # dim=-1로 마지막 차원인 tar_vocab_size에 대해 최대가 되는 값의 인덱스가 decoder_target의 인덱스와 같은 경우
            total_count += mask.sum().item() # 패딩 토큰을 제외한 개수

    return total_loss / len(dataloader), total_correct / total_count

In [24]:
encoder_input_train_tensor = torch.tensor(encoder_input_train, dtype=torch.long)
decoder_input_train_tensor = torch.tensor(decoder_input_train, dtype=torch.long)
decoder_target_train_tensor = torch.tensor(decoder_target_train, dtype=torch.long)

encoder_input_test_tensor = torch.tensor(encoder_input_test, dtype=torch.long)
decoder_input_test_tensor = torch.tensor(decoder_input_test, dtype=torch.long)
decoder_target_test_tensor = torch.tensor(decoder_target_test, dtype=torch.long)

# 데이터셋 및 데이터로더 생성
batch_size = 128

train_dataset = TensorDataset(encoder_input_train_tensor, decoder_input_train_tensor, decoder_target_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = TensorDataset(encoder_input_test_tensor, decoder_input_test_tensor, decoder_target_test_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [25]:
# 학습 설정
num_epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # 훈련 모드
    model.train()

    total_train_loss = 0.0
    total_train_correct = 0
    total_train_count = 0
    for encoder_inputs, decoder_inputs, decoder_targets in train_dataloader:
        encoder_inputs = encoder_inputs.to(device)
        decoder_inputs = decoder_inputs.to(device)
        decoder_targets = decoder_targets.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 순방향 전파
        # outputs.shape == (batch_size, seq_len, tar_vocab_size)
        outputs = model(encoder_inputs, decoder_inputs)

        # 손실 계산 및 역방향 전파
        # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
        # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
        loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

        # train_loss 및 accuracy 계산
        total_train_loss += loss.item()
        mask = decoder_targets != 0  # 패딩 토큰 제외
        total_train_correct += ((outputs.argmax(dim=-1) == decoder_targets) * mask).sum().item()
        total_train_count += mask.sum().item()

    # train_loss, train_acc = evaluation(model, train_dataloader, loss_function, device) # 기존의 코드는 train_dataloader를 한 epoch당 두 번 조회하는 문제가 있다
    train_loss = total_train_loss / len(train_dataloader)
    train_acc = total_train_correct / total_train_count
    valid_loss, valid_acc = evaluation(model, valid_dataloader, loss_function, device)

    print(f'Epoch: {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if valid_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {valid_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = valid_loss
        torch.save(model.state_dict(), 'best_model_checkpoint.pth')

Epoch: 1/30 | Train Loss: 3.9382 | Train Acc: 0.4314 | Valid Loss: 3.0173 | Valid Acc: 0.5246
Validation loss improved from inf to 3.0173. 체크포인트를 저장합니다.
Epoch: 2/30 | Train Loss: 2.6273 | Train Acc: 0.5687 | Valid Loss: 2.4859 | Valid Acc: 0.5911
Validation loss improved from 3.0173 to 2.4859. 체크포인트를 저장합니다.
Epoch: 3/30 | Train Loss: 2.1317 | Train Acc: 0.6198 | Valid Loss: 2.1835 | Valid Acc: 0.6266
Validation loss improved from 2.4859 to 2.1835. 체크포인트를 저장합니다.
Epoch: 4/30 | Train Loss: 1.7875 | Train Acc: 0.6560 | Valid Loss: 1.9916 | Valid Acc: 0.6475
Validation loss improved from 2.1835 to 1.9916. 체크포인트를 저장합니다.
Epoch: 5/30 | Train Loss: 1.5176 | Train Acc: 0.6883 | Valid Loss: 1.8502 | Valid Acc: 0.6656
Validation loss improved from 1.9916 to 1.8502. 체크포인트를 저장합니다.
Epoch: 6/30 | Train Loss: 1.2885 | Train Acc: 0.7215 | Valid Loss: 1.7307 | Valid Acc: 0.6825
Validation loss improved from 1.8502 to 1.7307. 체크포인트를 저장합니다.
Epoch: 7/30 | Train Loss: 1.0851 | Train Acc: 0.7542 | Valid Loss: 

In [26]:
# 모델 로드
model.load_state_dict(torch.load('best_model_checkpoint.pth'))

# 모델을 device에 올립니다.
model.to(device)

# 검증 데이터에 대한 정확도와 손실 계산
val_loss, val_accuracy = evaluation(model, valid_dataloader, loss_function, device)

print(f'Best model validation loss: {val_loss:.4f}')
print(f'Best model validation accuracy: {val_accuracy:.4f}')

Best model validation loss: 1.4729
Best model validation accuracy: 0.7256


<ipython-input-26-ccccf4da9b07>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_checkpoint.pth'))


In [27]:
print(tar_vocab['<sos>'])
print(tar_vocab['<eos>'])

3
4


## Machine Translation (Inference)

In [28]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0): # <PAD> 토큰은 제외
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_vocab['<sos>'] and encoded_word != tar_vocab['<eos>']): # <PAD>, <sos>, <eos>는 제외
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [29]:
print(encoder_input_test[25])
print(decoder_input_test[25])
print(decoder_target_test[25])

[   9   15   10 1949    5    0    0]
[   3   39  183   14   37   33 1126    7    0    0    0    0    0    0
    0    0]
[  39  183   14   37   33 1126    7    4    0    0    0    0    0    0
    0    0]


In [30]:
def decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, max_output_len, int_to_src_token, int_to_tar_token):
    # input_seq: 입력 문장을 정수 인코딩한 시퀀스/ input_seq.shape = [seq_length]
    encoder_inputs = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)
    # encoder_inputs.shape == [batch_size, sequence_length]이고 batch_size는 1이 된다

    # 인코더의 초기 상태 설정
    hidden, cell = model.encoder(encoder_inputs)

    # 시작 토큰 <sos>을 디코더의 첫 입력으로 설정
    # unsqueeze(0)는 배치 차원을 추가하기 위함.
    decoder_input = torch.tensor([3], dtype=torch.long).unsqueeze(0).to(device) # [[3]]의 형태로 들어가게 됨(shape = [batch_size, 1])

    decoded_tokens = []

    # for문을 도는 것 == 디코더의 각 시점
    for _ in range(max_output_len):
        output, hidden, cell = model.decoder(decoder_input, hidden, cell)

        # 소프트맥스 회귀를 수행. 예측 단어의 인덱스
        output_token = output.argmax(dim=-1).item()

        # 종료 토큰 <eos>
        if output_token == 4:
            break

        # 각 시점의 단어(정수)는 decoded_tokens에 누적하였다가 최종 번역 시퀀스로 리턴합니다.
        decoded_tokens.append(output_token)

        # 현재 시점의 예측. 다음 시점의 입력으로 사용된다.
        decoder_input = torch.tensor([output_token], dtype=torch.long).unsqueeze(0).to(device)

    return ' '.join(int_to_tar_token[token] for token in decoded_tokens)

In [31]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index]
  translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",translated_text)
  print("-"*50)

입력문장 : i m a humble guy . 
정답문장 : je suis un type humble . 
번역문장 : je suis un mec intelligent .
--------------------------------------------------
입력문장 : that ll be my job . 
정답문장 : ce sera mon travail . 
번역문장 : ce sera mon travail .
--------------------------------------------------
입력문장 : just be happy . 
정답문장 : sois tout simplement heureuse . 
번역문장 : soyez tout simplement heureux .
--------------------------------------------------
입력문장 : i m always hungry . 
정답문장 : j ai tout le temps faim . 
번역문장 : j ai toujours faim .
--------------------------------------------------
입력문장 : did tom fall ? 
정답문장 : tom est il tombe ? 
번역문장 : tom est il tombe ?
--------------------------------------------------


In [32]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index]
  translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",translated_text)
  print("-"*50)

입력문장 : go wait outside . 
정답문장 : va attendre dehors . 
번역문장 : allez attraper dehors !
--------------------------------------------------
입력문장 : tom is staying . 
정답문장 : tom reste . 
번역문장 : tom hurla .
--------------------------------------------------
입력문장 : you re charming . 
정답문장 : tu es charmant . 
번역문장 : vous etes charmante .
--------------------------------------------------
입력문장 : do you know us ? 
정답문장 : nous connaissez vous ? 
번역문장 : tu connais nous ?
--------------------------------------------------
입력문장 : get out of the car . 
정답문장 : sors de la voiture ! 
번역문장 : sortez de la camionnette !
--------------------------------------------------
